In [1]:
# Notebook header

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from pybat import Cathode
from pybat.workflow.workflows import get_wf_configurations

from vscworkflows.config import load_config

# Li-configuration - workflow

In order to study the stability of the Li2MnO3 structure in a charged state, we need to know the configuration of the remaining Li atoms. For the fully charged structure, this is not an issue, but this structure is found to be highly unstable, and hence difficult to study. In this notebook we set up the workflows for calculating the energy of all non-symmetric Li-configurations in both the O3 and O1 stacking of Li0.5MnO3.

## O3 Stacking

For the O3 stacking, we convert the structure to the conventional unit cell, and consider all non-symmetric Li configurations in this unit cell.

In [2]:
cat = Cathode.from_file("o3-abc-fm_Li2MnO3_prim.json")
conv = SpacegroupAnalyzer(cat).get_conventional_standard_structure()
conv.add_site_property("magmom", 8*[0] + 4*[3] + 12*[0])
cat_conv = Cathode.from_structure(conv)

In [3]:
confs = cat_conv.get_cation_configurations(
    substitution_sites=[i for i, s in enumerate(cat_conv) if s.species_string == "Li"],
    cation_list=["Li", "Vac"],
    sizes=[1],
#     concentration_restrictions={"Li": [0.08, 0.085]},
)

In [4]:
len(confs)

88

In [5]:
lpad = load_config("launchpad", "leibniz")
wf_dir = "/scratch/antwerpen/202/vsc20248/batteries/Li2MnO3/o3-fm"

In [6]:
wf = get_wf_configurations(
    structure=cat_conv,
    directory=wf_dir,
    configuration_list=confs,
    functional=("pbeu", {"LDAUU": {"Mn": 3.9}, "KPAR": 4, "NPAR": 4}),
)

In [7]:
lpad.add_wf(wf)

2019-11-28 01:43:32,191 INFO Added a workflow. id_map: {-1: 141}


{-1: 141}

## O1 stacking

For the O1 stacking, the conventional unit cell is rather small, and hence does not allow for a significant number of configurations. We therefore allow the `get_cation_configurations` script to generate unit cells up to size two, i.e. with two times the number of atoms in the conventional unit cell.

In [7]:
o1_cat = Cathode.from_file("o1-aaa-fm_Li2MnO3_prim.json")
o1_cat = Cathode.from_structure(SpacegroupAnalyzer(o1_cat).get_conventional_standard_structure())
o1_cat.add_site_property("magmom", 4*[0]+2*[3]+6*[0])

In [8]:
confs = o1_cat.get_cation_configurations(
    substitution_sites=list(range(4)),
    cation_list=["Li", "Vac"],
    sizes=[1, 2],
#     concentration_restrictions={"Li": [0.08, 0.085]},
)

In [9]:
len(confs)

208

In [10]:
lpad = load_config("launchpad", "leibniz")
wf_dir = "/scratch/antwerpen/202/vsc20248/batteries/Li2MnO3/o1-fm"

In [11]:
wf = get_wf_configurations(
    structure=o1_cat,
    directory=wf_dir,
    configuration_list=confs,
    functional=("pbeu", {"LDAUU": {"Mn": 3.9}, "KPAR": 4, "NPAR": 4}),
)

In [13]:
lpad.add_wf(wf)

2019-11-28 01:49:13,472 INFO Added a workflow. id_map: {-2: 230}


{-2: 230}